# Ejercicio

¿Cómo modelarían una base de datos que permitiera almacenar precios de mercado? Debiera poder gestionar precios de acciones, de instrumentos de renta fija, de swaps y quizás muchos más. ¿Cómo diseñar la BBDD de modo que pueda incorporar nuevos tipos de instrumentos?

Responder definiendo las distintas tablas con sus campos y relaciones. Pueden usar alguna de las herramientas GUI que les mostré o alguna otra que hayan encontrado.

## ¿Qué datos quiero guardar?

- Fecha, nombre, dato
- Si es tasa, precio, volatilidad
- Clase de activo (asset class de BBG)
- Hay activos que son instrumentos (acciones, bonos) y hay índices (Libor, el dólar observado).
- Hay mercado organizado o bursátil y mercado OTC.
- Plazo en algunos casos (curvas,) ---> según el activo voy a necesitar otros datos adicionales. Ejemplo, bono: tasa cupón, plazo, ...., futuro: settlement date ...
- Open, High, Low, Close, Volumen
- Convenciones si son tasas (day count, lineal o compuesto)
- Identificador del activo (nemo, ticker, cusip, ...)
- Fuente del precio --> características de la fuente ...
- Timestamp, bid, ask, last (relacionado a un timeframe)

## ¿Qué tablas defino?

- Serie de precios (precios diarios):
 - timestamp
 - identificador del activo (FK a Activos)
 - tipo de dato (FK al codigo de tipo de dato)
 - tipo de valor (bid, ask, close, last ..., NAV que aplica para los ETF)
 - valor
 
 
 - Tipo de valor
  - id
  - codigo (bid, ask, ...)
 
 
 - Tipo de dato
  - id
  - codigo (unique)
  - descripcion
 
 
 - Activos
  - id
  - codigo
  - codigo asset class
  - descripcion
  
  
- Caracteristicas Activos
 - id
 - codigo del activo (FK a Activos)
 - characteristicas: y esto es un JSON (piensen en un Dict de Python)


- Asset Class
 - id
 - codigo
 - descripcion


- Curvas - Superficies
 - id
 - codigo
 - descripcion


- Elementos Curvas
 - id
 - codigo activo
 - codigo curva
 

- Emisores


- Ratings
 - Necesita una dimensión temporal
 
 
------------------------------ 
**Descartado**

 - Factor de riesgo:
  - id
  - moneda
  - nombre del activo
  - id tipo factor
  - clave_isin (campo que sirve para meter más detalle)
  

 - Factor - Tipo
  - Indica el asset class del factor
  - 1 a 4 (tasa, spread, tipo de cambio, ...)
  
**Observaciones**

- FK a campos descriptivos y no a los id.

## ¿Qué queremos hacer con estos datos?

- Valorización de cierre de día
- Estadísticas
- Diseño y Backtesting de estrategias
- Métricas de riesgo: VaR ---> Tablas de retornos diarios
- Métricas de liquidez

## Campo `JSON`

SQLite no tiene JSON. ¿Cómo podemos emularlo?

https://www.w3schools.com/whatis/whatis_json.asp

In [1]:
import json

In [3]:
ejemplo = [
    {
        'nemo': 'bono1',
        'tir': .01
    },
    {
        'nemo': 'bono2',
        'tir': .02
    }
]

In [4]:
json_obj = json.dumps(ejemplo)

In [5]:
json_obj

'[{"nemo": "bono1", "tir": 0.01}, {"nemo": "bono2", "tir": 0.02}]'

In [6]:
json_str = str(json_obj)

In [7]:
json_str

'[{"nemo": "bono1", "tir": 0.01}, {"nemo": "bono2", "tir": 0.02}]'

Para insertar un JSON en SQLite, hay que agregar un campo de tipo TEXT y definir el json como un `dict` o `list` de `dict`, luego transformar a JSON con `json.dumps()` e insertar `str(json_object)`. Probar a insertar directamente `json_object`, puede que funcione.